In [3]:
#clone fusion_twas library before running this
!git clone https://github.com/gusevlab/fusion_twas.git

fatal: destination path 'fusion_twas' already exists and is not an empty directory.


In [3]:
!hostname

cn4245


In [1]:
import subprocess
import sys
import os
import shutil
import pandas as pd

In [2]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

In [3]:
#set paths
basedir = '/data/songy4/twas'
datadir = f'{basedir}/data_folder'
fusiondir = f'{basedir}/fusion_twas'
geno_path = f'{datadir}/qc_genotypes_twas'
gene_list_path = f'{datadir}/gene_list.txt'
pheno_path = f'{datadir}/expression_matrix_final.txt'
coord_path = f'{datadir}/twas_coordinate.txt'
covar_path = f'{datadir}/covariates.txt'
gcta = f'{fusiondir}/gcta_nr_robust'
gemma = f'gemma-0.98.3-linux-static'
fusion_ldref_basename = f'{fusiondir}/LDREF/1000G.EUR'
fusion_compute_weights_script = f'{fusiondir}/FUSION.compute_weights.R'

!mkdir --parents output/weights
!mkdir --parents output/tmp


# Pipeline for TWAS run

In [4]:
# get gene list
##test with couple genes first before running all gene_list
gene_list_df = pd.read_csv(gene_list_path, sep='\t')
gene_list = list(gene_list_df.ID)
# gene_list = ['ENSG00000186092', 'ENSG00000187634', 'ENSG00000188976'] # for testing
# gene_list = ['ENSG00000186092']
pheno = pd.read_csv(pheno_path, sep='\t')
coords = pd.read_csv(coord_path, sep='\t')
covars = pd.read_csv(covar_path, sep='\t')

In [5]:
# now put together pipeline

compweights_swarmfile = f'{basedir}/compute_weights.swarm'

with open(compweights_swarmfile, 'w') as f:
    
    for gene in gene_list:
        OUT = f'output/tmp/{gene}'
        FINAL_OUT = f'output/weights/{gene}'
        #get chr start stop
        _chr = coords.loc[coords.ID == gene, 'X.Chr'].item()
        _start = coords.loc[coords.ID == gene, 'start'].item()-0.5e6
        _stop = coords.loc[coords.ID == gene, 'end'].item()+0.5e6

        if _start < 0:
            _start = 0
        
        _start = int(_start)
        _stop = int(_stop)
        
#         _temp_name = f'{gene}_temp'
#         _gene_temp = f'{tempdir}/{_temp_name}'
        
        # pheno per gene
#         _phenoname = f'{_gene_temp}.pheno'
        
        # write pheno file per gene
        pheno[['FID','IID', gene]].to_csv(f'{OUT}.pheno', sep='\t', header=False, index=False)
#         pheno[['FID','IID', gene]].to_csv(_phenoname, sep='\t', header=False, index=False)
        
        plink_cmd = f'\
plink --bfile {geno_path}_hg19_lifted \
--pheno {OUT}.pheno \
--keep {OUT}.pheno \
--chr {_chr} \
--from-bp {_start} \
--to-bp {_stop} \
--extract {fusion_ldref_basename}.{_chr}.bim \
--make-bed \
--out {OUT}'  

#        shell_do(plink_cmd, log=True, return_log=True)

        fusion_cmd = f'\
Rscript {fusion_compute_weights_script} \
--bfile {OUT} \
--tmp {OUT}.tmp \
--out {FINAL_OUT} \
--PATH_gemma {gemma} \
--PATH_plink plink \
--PATH_gcta {gcta} \
--covar {covar_path} \
--verbose 2 \
--save_hsq \
--models top1,lasso,enet'
##include "blup,bslmm" to the --models in fusion_cmd for future     
# 
#        shell_do(fusion_cmd, log=True, return_log=True)

        f.write(f'{plink_cmd} && {fusion_cmd}\n')
    f.close()

In [6]:
# run swarm
swarm_cmd = f'swarm -f {compweights_swarmfile} -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm'
# shell_do(swarm_cmd)
print(swarm_cmd)
!{swarm_cmd}
##check swarm jobs state in hpc.nih.gov/dashboard/
##check .e, .o, and .log files of swarm jobs in swarm folder (same name as swarm jobid in hpc dashboard)

swarm -f /data/songy4/twas/compute_weights.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm
8041450


In [ ]:
#check number of weights in output/weights folder 
###run this in terminal

#go to the weights folder
!cd /data/songy4/twas/output/weights
#check how many files are in total
! ls | wc -l
#there are 27864
#check how many .RDat file is in weights folder
!ls -lR ./*.RDat | wc -l
#there are 9723 RDat files

TWAS

In [12]:
#open gene_chr_start_stop.tab -- this has chr and position info
chr_pos_df = pd.read_csv(coord_path, sep='\t')
chr_pos_df

,X.Chr,start,end,ID
0,1,69091,70008,ENSG00000186092
1,1,860260,879955,ENSG00000187634
2,1,879584,894689,ENSG00000188976
3,1,895967,901095,ENSG00000187961
4,1,901877,911245,ENSG00000187583
...,...,...,...,...
18236,21,47720095,47743789,ENSG00000160298
18237,21,47744036,47865682,ENSG00000160299
18238,21,47878812,47989926,ENSG00000160305
18239,21,48018875,48025121,ENSG00000160307


In [13]:
#rearrange the columns 
chr_pos_df = chr_pos_df[['ID', 'X.Chr','start','end']]
chr_pos_df.head()

,ID,X.Chr,start,end
0,ENSG00000186092,1,69091,70008
1,ENSG00000187634,1,860260,879955
2,ENSG00000188976,1,879584,894689
3,ENSG00000187961,1,895967,901095
4,ENSG00000187583,1,901877,911245


In [14]:
#change column names in chr_pos_df
chr_pos_df.columns = ['ID', 'CHR', 'P0', 'P1']
chr_pos_df

,ID,CHR,P0,P1
0,ENSG00000186092,1,69091,70008
1,ENSG00000187634,1,860260,879955
2,ENSG00000188976,1,879584,894689
3,ENSG00000187961,1,895967,901095
4,ENSG00000187583,1,901877,911245
...,...,...,...,...
18236,ENSG00000160298,21,47720095,47743789
18237,ENSG00000160299,21,47744036,47865682
18238,ENSG00000160305,21,47878812,47989926
18239,ENSG00000160307,21,48018875,48025121


In [7]:
#make gene_path.csv
! ls -l ./output/weights | grep -E 'ENSG|.RDat' | awk '{print $9", ./weights/"$9}' | sed 's/.wgt.RDat,/,/g' > ./data_folder/gene_path.csv

## list files in weights folder, search lines with same pattern as 'ENSG'
## scan file line by line, splits each input line
## substitute .wgt.RDat, with , in all occurrences (/g means global replacement)

In [8]:
#import gene_path.csv 
path = pd.read_csv(f"{datadir}/gene_path.csv", names=['ID', 'WGT'])
path.head

<bound method NDFrame.head of                         ID                                  WGT
0      ENSG00000000419.hsq        ./weights/ENSG00000000419.hsq
1      ENSG00000000457.hsq        ./weights/ENSG00000000457.hsq
2          ENSG00000000457   ./weights/ENSG00000000457.wgt.RDat
3      ENSG00000000460.hsq        ./weights/ENSG00000000460.hsq
4          ENSG00000000460   ./weights/ENSG00000000460.wgt.RDat
...                    ...                                  ...
27859  ENSG00000273213.hsq        ./weights/ENSG00000273213.hsq
27860  ENSG00000273217.hsq        ./weights/ENSG00000273217.hsq
27861  ENSG00000273238.hsq        ./weights/ENSG00000273238.hsq
27862  ENSG00000273274.hsq        ./weights/ENSG00000273274.hsq
27863  ENSG00000273291.hsq        ./weights/ENSG00000273291.hsq

[27864 rows x 2 columns]>

In [9]:
#remove .hsq rows
path = path[~path.WGT.str.contains(".hsq")]
path.head

<bound method NDFrame.head of                     ID                                  WGT
2      ENSG00000000457   ./weights/ENSG00000000457.wgt.RDat
4      ENSG00000000460   ./weights/ENSG00000000460.wgt.RDat
7      ENSG00000000971   ./weights/ENSG00000000971.wgt.RDat
9      ENSG00000001036   ./weights/ENSG00000001036.wgt.RDat
11     ENSG00000001084   ./weights/ENSG00000001084.wgt.RDat
...                ...                                  ...
27838  ENSG00000272514   ./weights/ENSG00000272514.wgt.RDat
27842  ENSG00000272636   ./weights/ENSG00000272636.wgt.RDat
27848  ENSG00000273045   ./weights/ENSG00000273045.wgt.RDat
27855  ENSG00000273155   ./weights/ENSG00000273155.wgt.RDat
27858  ENSG00000273173   ./weights/ENSG00000273173.wgt.RDat

[9723 rows x 2 columns]>

In [10]:
#save path as csv -- run only once for download
path.to_csv(r'./data_folder/gene_path_1.csv' ,index=False)

In [15]:
#merge chr_pos_df and path
merged = chr_pos_df.merge(path, how='inner', on='ID')
merged.head

<bound method NDFrame.head of                    ID  CHR        P0        P1  \
0     ENSG00000188976    1    879584    894689   
1     ENSG00000187961    1    895967    901095   
2     ENSG00000187583    1    901877    911245   
3     ENSG00000188290    1    934342    935552   
4     ENSG00000187608    1    948803    949920   
...               ...  ...       ...       ...   
9718  ENSG00000160298   21  47720095  47743789   
9719  ENSG00000160299   21  47744036  47865682   
9720  ENSG00000160305   21  47878812  47989926   
9721  ENSG00000160307   21  48018875  48025121   
9722  ENSG00000160310   21  48055079  48085036   

                                      WGT  
0      ./weights/ENSG00000188976.wgt.RDat  
1      ./weights/ENSG00000187961.wgt.RDat  
2      ./weights/ENSG00000187583.wgt.RDat  
3      ./weights/ENSG00000188290.wgt.RDat  
4      ./weights/ENSG00000187608.wgt.RDat  
...                                   ...  
9718   ./weights/ENSG00000160298.wgt.RDat  
9719   ./weights/

In [16]:
#check chr 
chr_list = merged['CHR'].unique().tolist()
chr_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 19, 22, 21]

In [17]:
#drop ./weights/ from WGT
merged['WGT'] = merged['WGT'].str.replace('./weights/', '')
merged.head

<bound method NDFrame.head of                    ID  CHR        P0        P1                        WGT
0     ENSG00000188976    1    879584    894689   ENSG00000188976.wgt.RDat
1     ENSG00000187961    1    895967    901095   ENSG00000187961.wgt.RDat
2     ENSG00000187583    1    901877    911245   ENSG00000187583.wgt.RDat
3     ENSG00000188290    1    934342    935552   ENSG00000188290.wgt.RDat
4     ENSG00000187608    1    948803    949920   ENSG00000187608.wgt.RDat
...               ...  ...       ...       ...                        ...
9718  ENSG00000160298   21  47720095  47743789   ENSG00000160298.wgt.RDat
9719  ENSG00000160299   21  47744036  47865682   ENSG00000160299.wgt.RDat
9720  ENSG00000160305   21  47878812  47989926   ENSG00000160305.wgt.RDat
9721  ENSG00000160307   21  48018875  48025121   ENSG00000160307.wgt.RDat
9722  ENSG00000160310   21  48055079  48085036   ENSG00000160310.wgt.RDat

[9723 rows x 5 columns]>

In [18]:
#rearrange columns and save the file as synapse.pos
output = merged[['WGT', 'ID', 'CHR', 'P1', 'P0']]
output.to_csv(r'./data_folder/synapse.pos', index=False, sep="\t")

In [19]:
#set paths
sumstat_path = f'{datadir}/meta_1.txt'
weights_path = f'{datadir}/synapse.pos'
weights_dir = f'{basedir}/output/weights/'
fusion_ldref_basename = f'{fusiondir}/LDREF/1000G.EUR'
fusion_assoc_script = f'{fusiondir}/FUSION.assoc_test.R'

!mkdir --parents output/pd

sumstat = pd.read_csv(sumstat_path, sep='\t')

##check if weights_dir should be f'{basedir}/output/weights'####

In [20]:
#fusion calculation -- take about 3 min each chromosome 
for i in range(1, 23):
    PD_OUT = f'output/pd/PD.{i}.dat'
    fusion_cmd_i = f'\
    Rscript {fusion_assoc_script} \
    --sumstats {sumstat_path} \
    --weights {weights_path} \
    --weights_dir {weights_dir} \
    --ref_ld_chr {fusion_ldref_basename}. \
    --chr {i} \
    --out {PD_OUT}'
    shell_do(fusion_cmd_i)

Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/twas/data_folder/meta_1.txt --weights /data/songy4/twas/data_folder/synapse.pos --weights_dir /data/songy4/twas/output/weights/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 1 --out output/pd/PD.1.dat
Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/twas/data_folder/meta_1.txt --weights /data/songy4/twas/data_folder/synapse.pos --weights_dir /data/songy4/twas/output/weights/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 2 --out output/pd/PD.2.dat
Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/twas/data_folder/meta_1.txt --weights /data/songy4/twas/data_folder/synapse.pos --weights_dir /data/songy4/twas/output/weights/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 3 --out output/pd/PD.3.dat
Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.as

# running list of questions
1. what about non-autosome transcripts? i.e. MT
2. how many snps in common between ref and geno?
3. how many snps per transcript should we expect (0-500?)


In [44]:
p_in = 'tmp/ENSG00000188976.tmp.cv'
f"gemma -miss 1 -maf 0 -r2 1 -rpace 1000 -wpace 1000 -bfile {p_in} -bslmm 2 -o ENSG00000188976"

'gemma -miss 1 -maf 0 -r2 1 -rpace 1000 -wpace 1000 -bfile tmp/ENSG00000188976.tmp.cv -bslmm 2 -o ENSG00000188976'